In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [9]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [10]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [11]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [14]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "external"}
)

print(response["messages"][-1].content)

I don’t have access to your database, so I can’t see the exact count. If you tell me the database type and the table/collection name (and whether you want an exact or approximate count), I can give you the exact query. Here are quick options you can run yourself:

- Relational databases (MySQL, PostgreSQL, SQL Server, SQLite)
  - Exact count:
    SELECT COUNT(*) AS total_artists FROM artists;
  - If there’s a primary key like artist_id and you want to be sure:
    SELECT COUNT(DISTINCT artist_id) FROM artists;

- MongoDB
  - Exact count:
    db.artists.countDocuments({});

- Elasticsearch
  - Exact count:
    GET /artists/_count

- If you’re using a REST API
  - Check if the API returns total or metadata with the list, or try endpoints like /artists?limit=1 to infer total pages.

If you share:
- Database type
- Collection/table name (e.g., artists)
- Any filters (e.g., only active artists)
I’ll tailor the exact query for you and help you interpret the result.
